### Quickstart: Compare runs, choose a model, and deploy it to a REST API

In this notebook
- Run a hyperparameter sweep on a training script
- Compare the results of the runs in Mlflow ui
- Choose the best run and register it as a model
- Deploy the model to a REST API
- Build a container image suitable for deployment to a cloud platform

In [117]:
!pip install hyperopt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [118]:
import keras
import numpy as np
import pandas as pd

# Hyperopt is used for hyperparameter tuning for Deep Learning models.
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [119]:
## load the dataset
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [120]:
# Split the data into training, validation and test sets
train,test = train_test_split(data, test_size=0.2, random_state=42)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4665,7.3,0.17,0.36,8.20,0.028,44.0,111.0,0.99272,3.14,0.41,12.4,6
1943,6.3,0.25,0.44,11.60,0.041,48.0,195.0,0.99680,3.18,0.52,9.5,5
3399,5.6,0.32,0.33,7.40,0.037,25.0,95.0,0.99268,3.25,0.49,11.1,6
843,6.9,0.19,0.35,1.70,0.036,33.0,101.0,0.99315,3.21,0.54,10.8,7
2580,7.7,0.30,0.26,18.95,0.053,36.0,174.0,0.99976,3.20,0.50,10.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [121]:
train_X = train.drop("quality", axis=1).values
# .ravel is used to convert the DataFrame to a 1D array
train_y = train["quality"].values.ravel()

# Validation dataset 
test_x = test.drop("quality", axis=1).values
test_y = test["quality"].values.ravel()


# Split the training data into training and validation sets
train_X,valid_X,train_y,valid_y = train_test_split(
    train_X, train_y, test_size=0.2, random_state=42
)

signature = infer_signature(train_X, train_y)

In [122]:
# ANN model

import mlflow.tensorflow


def train_model(params,epochs,train_X,train_y,valid_X,valid_y,test_x,test_y):
    
    # Define the model architecture
    mean = np.mean(train_X,axis=0)
    var = np.var(train_X,axis=0)
    
    model = keras.Sequential(
        [
        keras.Input([train_X.shape[1]]),
        keras.layers.Normalization(mean=mean, variance=var),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dense(1)
        ]
    )
    
    model.compile(optimizer = keras.optimizers.SGD(
        learning_rate =params['lr'],
        momentum = params['momentum'],
    ),
    loss = "mean_squared_error",
    metrics = [keras.metrics.RootMeanSquaredError()])
    
    # Train the model
    
    with mlflow.start_run(nested=True):
        model.fit(train_X,train_y,validation_data=(valid_X,valid_y),
                epochs = epochs,
                batch_size= 64)
        
        # Evaluate the model on the test set
        eval_result = model.evaluate(valid_X,valid_y, batch_size=64)
        
        eval_rmse = eval_result[1]
        
        # log the parameter and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)
        
        # log the model
        mlflow.tensorflow.log_model(model,"model",
                                signature=signature,
                                )
        
        return {
            "loss": eval_rmse,
            "status": STATUS_OK,
            "model": model,
        }

In [123]:
def objective(params):
    # MLflow will track the parameters and results of each run
    result = train_model(
        params,
        epochs=3,
        train_X=train_X,
        train_y=train_y,
        valid_X=valid_X,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y
    )
    return result


In [124]:
space = {
    "lr" : hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum", 0.0, 0.9),
}

In [125]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model")  # signature removed

    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


Epoch 1/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 15s 330ms/step - loss: 35.6130 - root_mean_squared_error: 5.9677
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 21.7524 - root_mean_squared_error: 4.6009   
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 21.5673 - root_mean_squared_error: 4.5796 - val_loss: 3.2817 - val_root_mean_squared_error: 1.8116

Epoch 2/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 3.1878 - root_mean_squared_error: 1.7854
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.4911 - root_mean_squared_error: 1.5770 - val_loss: 2.2289 - val_root_mean_squared_error: 1.4929

Epoch 3/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 1.9977 - root_mean_squared_error: 1.4134
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.7310 - root_mean_squared_error: 1.3154 - val_loss: 1.8495 - val_root_mean_squared_error: 1.3599

 1/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms

2025/06/09 03:44:01 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


2025/06/09 03:44:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Best parameters: {'lr': 0.09240564863599812, 'momentum': 0.5340682838337641}
Best eval rmse: 0.7364414930343628


In [128]:
import mlflow

model_uri = 'runs:/4dd3db076e4e4ff48e5e7b2d075afdfb/model'

# Replace INPUT_EXAMPLE with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
input_data = test_x

# Verify the model with the provided input data using the logged dependencies.
# For more details, refer to:
# https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
mlflow.models.predict(
    model_uri=model_uri,
    input_data=input_data,
    env_manager="local",
)

2025/06/09 03:53:23 INFO mlflow.models.python_api: It is highly recommended to use `uv` as the environment manager for predicting with MLflow models as its performance is significantly better than other environment managers. Run `pip install uv` to install uv. See https://docs.astral.sh/uv/getting-started/installation for other installation methods.


2025/06/09 03:53:23 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
{"predictions": [[6.4486565589904785], [6.676247596740723], [6.604545593261719], [5.843067646026611], [6.292703628540039], [6.74277400970459], [5.890346050262451], [5.409409523010254], [6.071342468261719], [5.792107582092285], [6.638113975524902], [4.828298568725586], [6.895570755004883], [5.53580379486084], [7.446080684661865], [5.8251261711120605], [6.762073993682861], [6.278621673583984], [6.13313102722168], [5.825350761413574], [5.884515762329102], [6.194972038269043], [5.605582237243652], [6.581955432891846], [6.2860260009765625], [5.7660017013549805], [5.5777459144592285], [6.299234867095947], [6.270047187805176], [5.498980522155762], [5.840116500854492], [6.021317958831787], [5.877445220947266], [5.767542362213135], [5.934970378875732], [6.5357441902160645], [6.484499454498291], [5.349887371063232], [5.997633457183838], [6.0958967208862305], [6.125141620635986], [5.791303634643555], [6.018287658691406], [5.824871063232422], [5.2959680557250

In [131]:
import mlflow
logged_model = 'runs:/4dd3db076e4e4ff48e5e7b2d075afdfb/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
# Only use feature columns for prediction
feature_data = data.drop("quality", axis=1)
loaded_model.predict(feature_data)

154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 847us/step


,0
0,6.499022
1,5.337129
2,5.776067
3,6.086769
4,6.086769
...,...
4893,6.234409
4894,5.816775
4895,5.743985
4896,6.766796
